# Momento evaluativo 3

Seminario <br>
Especialización en Analítica y Ciencia de Datos <br>
Universidad de Antioquia <br>
Noviembre 2023 <br>

Julieth Tatiana García Zuluaga - Grupo 12

# Decripción del dataset

El siguiente dataset es una extracción de la base de datos del foro de discusión de una empresa. Sus campos se seleccionaron de acuerdo al objetivo de realizar una calidad de las respuestas proporcionadas mediante un análisis de sentimientos del mismo.

| Campo | Descripción | 
| :--- | :--- |
| id | ID propio de cada registro del post |
| raw | Contenido del post realizado en el foro |
| post_number | Número del post publicado |
| like_count | Cantidad de likes otorgados |
| score | Puntaje otorgado al post publicado |
| cooked | Versión procesada del post publicado |
| reads | Número de veces que el post ha sido leido por los usuarios |
| word_count | Cantidad de palabras que contiene el post |
| image_upload_id | ID de la imagen insertada (Si aplica) |
| post_action_type | Tipo de acción en el post |
| topic | Tema relacionado al post |
| views | Cantidad de visualizaciones que ha recibido el post |
| post_count | Cantidad de mensajes publicados en relación con el post |
| archetype | Arquetipo del post |
| excerpt | Extracción de post |
| category | Categoría a la que pertenece el post |
| category_description | Descripción de la categoría a la que pertenece el post |
| category_search_data | Categoría de búsqueda en el foro |
| tag | Etiqueta otorgada al post |

Resumen de las características:
- Instancias: 10.547
- Columnas: 19
- Objetivo: Evaluar la calidad de las respuestas de los usuarios en el foro,
***

# Objetivo

Evaluar la calidad de las respuestas proporcionadas por los usuarios en el foro de discusión en la Vicepresidencia de Tecnología de la empresa.

# Librerías y configuraciones previas

In [46]:
# Tratamiento de datos
import pandas as pd

# Corrección etiquetas HTML
from bs4 import BeautifulSoup
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
import nltk
nltk.download('stopwords')
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer


# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración de warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tatianagarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Funciones

In [47]:
# Función para limpiar etiquetas html de una columna de un dataset
#
# texto_html: columna del dataframe que contiene etiquetas html 
#
# Ejemplos:     
#    df['Texto_HTML'] = df['Texto_HTML'].apply(limpiar_html)
# ==============================================================================

# Función para limpiar etiquetas HTML
def limpiar_html(texto_html):
    # Utiliza el analizador HTML de BeautifulSoup
    soup = BeautifulSoup(texto_html, 'html.parser')

    # Extrae solo el texto de las etiquetas
    texto_limpio = ' '.join(soup.stripped_strings)

    return re.sub('\s+', ' ', texto_limpio)


def preprocess_text(text):
    # Convierte a minúsculas
    text = text.lower()
    # Tokenización
    tokens = word_tokenize(text)
    # Filtrar stopwords y signos de puntuación
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    return tokens



# Cargar dataset

In [48]:
# Vista preliminar del data set
data = pd.read_csv('https://raw.githubusercontent.com/TatianaGarcia1128/foro_discusion_udea/main/dataset_posts.csv')
data.head()

,id,raw,post_number,like_count,score,cooked,reads,word_count,image_upload_id,post_action_type,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag
0,152,"Donde, con quien y como puedo resolver dudas y...",1,0,5.8,"<p>Donde, con quien y como puedo resolver duda...",29,19,NaN,NaN,Como puedo Solicitar apoyo sobre el sistema de...,37.0,4.0,regular,"Donde, con quien y como puedo resolver dudas y...",NaN,NaN,NaN,NaN
1,48,La copia de respaldo ha fallado.\nAquí está el...,1,0,0.2,<p>La copia de respaldo ha fallado.</p>\n<p>Aq...,1,979,NaN,NaN,La copia de respaldo falló,0.0,1.0,private_message,La copia de respaldo ha fallado. \nAquí está e...,NaN,NaN,NaN,NaN
2,3539,"Gracias por unirte a Discuss Bancolombia, ¡y b...",1,0,0.2,"<p>Gracias por unirte a Discuss Bancolombia, ¡...",1,129,NaN,NaN,¡Saludos!,0.0,1.0,private_message,"Gracias por unirte a Discuss Bancolombia, ¡y b...",NaN,NaN,NaN,NaN
3,931,saltar,7,0,5.2,<p>saltar</p>,1,1,NaN,NaN,¡Saludos!,2.0,9.0,private_message,"Gracias por unirte a Bancolombia Tech, ¡y bien...",NaN,NaN,NaN,NaN
4,6,"<a name=""civilized""></a>\n\n## [Mejora la disc...",1,0,0.4,"<p><a name=""civilized""></a></p>\n<h2>\n<a name...",2,631,NaN,NaN,Preguntas frecuentes,1.0,2.0,regular,Mejora la discusion\n\npara mejorar la discusi...,NaN,NaN,NaN,NaN


El dataset contiene etiquetas html que deben tener un procesamiento previo a la aplicación de las técnicas de ML. De igual forma, se puiede ver que el 'archetype' private_messsage corresponde a mensajes del sistema que no aportan al objetivo de este estudio por lo que es posible eliminarlos.

In [49]:
# Revisar nulos, tipo de datos en cada variable y tamaño de la base de datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10547 non-null  int64  
 1   raw                   10247 non-null  object 
 2   post_number           10547 non-null  int64  
 3   like_count            10547 non-null  int64  
 4   score                 10547 non-null  float64
 5   cooked                10247 non-null  object 
 6   reads                 10547 non-null  int64  
 7   word_count            10547 non-null  int64  
 8   image_upload_id       2019 non-null   float64
 9   post_action_type      2546 non-null   object 
 10  topic                 10543 non-null  object 
 11  views                 10543 non-null  float64
 12  posts_count           10543 non-null  float64
 13  archetype             10543 non-null  object 
 14  excerpt               10250 non-null  object 
 15  category           

El dataset contiene 10547 instancias y 19 columnas, de las cuales 5 tienen valores tipo entero (int64), 4 de tipo decimal (float64) y las 10 restantes son tipo object. Presenta las siguientes columnas con valores nulos 'image_upload_id', 'post_action_type', 'raw', 'topic', 'views', 'posts_count', 'archetype', 'excerpt', 'category', 'category_description', 'category_search_data' y 'tag' y en total el peso del dataset es de 1.5+ MB; es decir, que es fácil de procesar.

In [50]:
# Estadística descriptiva de las variables númericas
data.describe()

,id,post_number,like_count,score,reads,word_count,image_upload_id,views,posts_count
count,10547.000000,10547.000000,10547.000000,10547.000000,10547.000000,10547.000000,2019.000000,10543.000000,10543.000000
mean,4295.824500,3.021618,0.838058,28.966341,16.948895,108.692424,1563.709757,33.041639,4.928958
std,2663.735105,4.616849,2.454312,75.896358,25.638879,166.350412,833.799621,76.131011,7.591295
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000
25%,1992.000000,1.000000,0.000000,0.400000,1.000000,37.000000,835.000000,1.000000,1.000000
50%,4265.000000,1.000000,0.000000,5.200000,8.000000,75.000000,1559.000000,14.000000,2.000000
75%,6660.500000,3.000000,0.000000,23.400000,23.000000,129.000000,2303.000000,41.000000,4.000000
max,8961.000000,39.000000,24.000000,905.000000,260.000000,3091.000000,2992.000000,884.000000,39.000000


En general las variables parecen tener una escala similar, de conteo de cada una de las características que hacen parte del foro. La variable 'score' es la que podía presentar un valor diferente.

# Limpieza de datos (Data Cleaning)

In [51]:
#Borrando columnas innecesarias
data.drop([ 'id', 'image_upload_id', 'post_action_type'], axis='columns', inplace=True)
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10547 entries, 0 to 10546
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   raw                   10247 non-null  object 
 1   post_number           10547 non-null  int64  
 2   like_count            10547 non-null  int64  
 3   score                 10547 non-null  float64
 4   cooked                10247 non-null  object 
 5   reads                 10547 non-null  int64  
 6   word_count            10547 non-null  int64  
 7   topic                 10543 non-null  object 
 8   views                 10543 non-null  float64
 9   posts_count           10543 non-null  float64
 10  archetype             10543 non-null  object 
 11  excerpt               10250 non-null  object 
 12  category              611 non-null    object 
 13  category_description  221 non-null    object 
 14  category_search_data  611 non-null    object 
 15  tag                

In [52]:
#Eliminar categoria que no aporta al modelo
data = data[data['archetype'] != 'private_message']
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5719 entries, 0 to 10546
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   raw                   5672 non-null   object 
 1   post_number           5719 non-null   int64  
 2   like_count            5719 non-null   int64  
 3   score                 5719 non-null   float64
 4   cooked                5672 non-null   object 
 5   reads                 5719 non-null   int64  
 6   word_count            5719 non-null   int64  
 7   topic                 5715 non-null   object 
 8   views                 5715 non-null   float64
 9   posts_count           5715 non-null   float64
 10  archetype             5715 non-null   object 
 11  excerpt               5422 non-null   object 
 12  category              611 non-null    object 
 13  category_description  221 non-null    object 
 14  category_search_data  611 non-null    object 
 15  tag                   601

In [53]:
# Verificar registros duplicados
data.loc[data.duplicated()]

,raw,post_number,like_count,score,cooked,reads,word_count,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag
54,les comparto el DoR de la célula que atiende l...,3,2,32.6,<p>les comparto el DoR de la célula que atiend...,13,38,HA para solicitar aprobación para repositorio ...,20.0,3.0,regular,Cordial saludo. \nQuiero saber cual es el proc...,NaN,NaN,NaN,NaN
97,"Hola Carlos, eso se debio a una actualización ...",2,2,70.0,"<p>Hola Carlos, eso se debio a una actualizaci...",25,22,Fallo Stage IaC Scan - dev,31.0,3.0,regular,"Buenos dias, desde el sábado se está presentan...",NaN,NaN,NaN,NaN
117,¿Cómo debo configurar mi stage de pruebas de E...,1,2,56.8,<p>¿Cómo debo configurar mi stage de pruebas d...,34,11,Pruebas de Performance E2E - Performance E2E -...,40.0,2.0,regular,¿Cómo debo configurar mi stage de pruebas de E...,NaN,NaN,NaN,NaN
197,(Sustituye este párrafo con una descripción co...,1,0,0.0,<p>(Sustituye este párrafo con una descripción...,0,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,Debido a optimizaciones y un proceso de simpli...,1,6,237.6,<p>Debido a optimizaciones y un proceso de sim...,39,126,Si ejecutas pruebas de performance por el pool...,59.0,4.0,regular,Debido a optimizaciones y un proceso de simpli...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10520,"Concuerdo con la respuesta de @rsmenese , pued...",3,4,63.8,"<p>Concuerdo con la respuesta de <a class=""men...",20,56,Actualización de Hora en componente Page_Header,88.0,3.0,regular,"Hola, Quisiera saber si para el componente BCP...",NaN,NaN,NaN,NaN
10521,"Concuerdo con la respuesta de @rsmenese , pued...",3,4,63.8,"<p>Concuerdo con la respuesta de <a class=""men...",20,56,Actualización de Hora en componente Page_Header,88.0,3.0,regular,"Hola, Quisiera saber si para el componente BCP...",NaN,NaN,NaN,NaN
10531,"Buenas tardes @sebza,\n\nRevisando el error qu...",2,2,72.6,"<p>Buenas tardes <a class=""mention"" href=""/u/s...",39,111,Error en pruebas E2E - There is not enough spa...,66.0,2.0,regular,"Buenas tardes, me sale un error en el gradlew ...",NaN,NaN,NaN,NaN
10534,Hola @kevalenc es importante tener presente qu...,2,2,31.6,"<p>Hola <a class=""mention"" href=""/u/kevalenc"">...",8,247,Consultar registro de logs en el visor de dato...,12.0,2.0,regular,"Hola buenas tardes, debo realizar un conteo de...",NaN,NaN,NaN,NaN


Se encuentran un total de 3.991 registros duplicados, los cuales se deciden eliminar ya que los post repetitivos pueden ser mensajes del sistema o preguntas repetitivas por parte de los usuarios.

In [54]:
#Eliminar valores duplicados
data = data.drop_duplicates()

In [55]:
nulos_raw = data[data['raw'].isnull()]
nulos_raw

,raw,post_number,like_count,score,cooked,reads,word_count,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag
1078,NaN,2,0,3.4,NaN,18,0,Inconvenientes en despliegues PleXo,31.0,3.0,regular,Buenas tardes. \nPor un inconveniente transver...,NaN,NaN,NaN,NaN
1087,NaN,2,0,2.0,NaN,10,0,Acerca de la categoría Skill Hacking,28.0,2.0,regular,(Sustituye este párrafo con una descripción co...,Skill Hacking,NaN,Skill Hacking,plan-de-formacion
2814,NaN,2,0,0.2,NaN,1,0,Acerca de la categoría Comunicados,31.0,1.0,regular,NaN,Comunicados,NaN,Comunicados,continuous-hacking
3295,NaN,5,0,7.4,NaN,37,0,Error de Certificados CERTIFICATE_VERIFY_FAILE...,104.0,6.0,regular,Recientemente mi laptop MAC fue formateado y a...,NaN,NaN,NaN,NaN
3381,NaN,4,0,2.8,NaN,14,0,[Error] invalid character '\'' looking for beg...,27.0,4.0,regular,"En el pipeline de build, hay un error extraño ...",NaN,NaN,NaN,NaN
4201,NaN,3,0,0.2,NaN,1,0,Acerca de la categoría Arquitectura de TI,53.0,1.0,regular,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,pipeline-release
4202,NaN,3,0,0.2,NaN,1,0,Acerca de la categoría Arquitectura de TI,53.0,1.0,regular,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,java
4203,NaN,3,0,0.2,NaN,1,0,Acerca de la categoría Arquitectura de TI,53.0,1.0,regular,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,vulnerability
4204,NaN,3,0,0.2,NaN,1,0,Acerca de la categoría Arquitectura de TI,53.0,1.0,regular,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,aws
4205,NaN,3,0,0.2,NaN,1,0,Acerca de la categoría Arquitectura de TI,53.0,1.0,regular,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,Queremos en este Blog compartir temas actuales...,Arquitectura de TI,engine-iac


De acuerdo con esta consulta de los valores nulos asociados a una de las variables de interés 'raw', estos resultados corresponde a mensajes privados del sistema relacionados con la copia de seguridad; por lo que pueden ser eliminados.

In [56]:
data.dropna(subset=['raw'], inplace=True)

In [57]:
# Verificar registros duplicados
data.loc[data.duplicated()]

,raw,post_number,like_count,score,cooked,reads,word_count,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag


In [58]:
# Reemplazar los valores nulos con 'No aplica' en las columnas
data['category'] = data['category'].fillna('No aplica')
data['category_description'] = data['category_description'].fillna('No aplica')
data['category_search_data'] = data['category_search_data'].fillna('No aplica')
data['tag'] = data['tag'].fillna('No aplica')
data['excerpt'] = data['excerpt'].fillna('No aplica')
data['cooked'] = data['cooked'].fillna('No aplica')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4474 entries, 0 to 10546
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   raw                   4474 non-null   object 
 1   post_number           4474 non-null   int64  
 2   like_count            4474 non-null   int64  
 3   score                 4474 non-null   float64
 4   cooked                4474 non-null   object 
 5   reads                 4474 non-null   int64  
 6   word_count            4474 non-null   int64  
 7   topic                 4472 non-null   object 
 8   views                 4472 non-null   float64
 9   posts_count           4472 non-null   float64
 10  archetype             4472 non-null   object 
 11  excerpt               4474 non-null   object 
 12  category              4474 non-null   object 
 13  category_description  4474 non-null   object 
 14  category_search_data  4474 non-null   object 
 15  tag                   447

In [59]:
nulos_topic = data[data['topic'].isnull()]
nulos_topic

,raw,post_number,like_count,score,cooked,reads,word_count,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag
90,(Sustituye este párrafo con una descripción co...,1,0,0.0,<p>(Sustituye este párrafo con una descripción...,0,87,NaN,NaN,NaN,NaN,No aplica,No aplica,No aplica,No aplica,No aplica
1493,(Sustituye este párrafo con una descripción co...,1,0,0.2,<p>(Sustituye este párrafo con una descripción...,1,87,NaN,NaN,NaN,NaN,No aplica,No aplica,No aplica,No aplica,No aplica


Es posible eliminar estos valores nulos ya que no contienen información relevante para nuestro modelo.

In [60]:
data.dropna(subset=['topic'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4472 entries, 0 to 10546
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   raw                   4472 non-null   object 
 1   post_number           4472 non-null   int64  
 2   like_count            4472 non-null   int64  
 3   score                 4472 non-null   float64
 4   cooked                4472 non-null   object 
 5   reads                 4472 non-null   int64  
 6   word_count            4472 non-null   int64  
 7   topic                 4472 non-null   object 
 8   views                 4472 non-null   float64
 9   posts_count           4472 non-null   float64
 10  archetype             4472 non-null   object 
 11  excerpt               4472 non-null   object 
 12  category              4472 non-null   object 
 13  category_description  4472 non-null   object 
 14  category_search_data  4472 non-null   object 
 15  tag                   447

### Limpieza de etiquetas html

Se aplica una limpieza de etiquetas html en las columnas: raw y cooked.

In [61]:
# Aplicar la función a la columna 'Texto_HTML'
data['raw'] = data['raw'].apply(limpiar_html)
data['cooked'] = data['cooked'].apply(limpiar_html)
data.head()

,raw,post_number,like_count,score,cooked,reads,word_count,topic,views,posts_count,archetype,excerpt,category,category_description,category_search_data,tag
0,"Donde, con quien y como puedo resolver dudas y...",1,0,5.8,"Donde, con quien y como puedo resolver dudas y...",29,19,Como puedo Solicitar apoyo sobre el sistema de...,37.0,4.0,regular,"Donde, con quien y como puedo resolver dudas y...",No aplica,No aplica,No aplica,No aplica
4,## [Mejora la discusion](#mejora) para mejorar...,1,0,0.4,Mejora la discusion para mejorar la discusión ...,2,631,Preguntas frecuentes,1.0,2.0,regular,Mejora la discusion\n\npara mejorar la discusi...,No aplica,No aplica,No aplica,No aplica
7,These terms govern use of the Internet forum a...,1,0,0.2,These terms govern use of the Internet forum a...,1,2337,Terms of Service,0.0,2.0,regular,These terms govern use of the Internet forum a...,No aplica,No aplica,No aplica,No aplica
8,Edit the first post in this topic to change th...,2,0,0.2,Edit the first post in this topic to change th...,1,17,Terms of Service,0.0,2.0,regular,These terms govern use of the Internet forum a...,No aplica,No aplica,No aplica,No aplica
15,## [What information do we collect?](#collect)...,1,0,0.2,What information do we collect? We collect inf...,1,806,Privacy Policy,0.0,2.0,regular,What information do we collect?\n\nWe collect ...,No aplica,No aplica,No aplica,No aplica


# Preprocesamiento de texto y creación de bolsa de palabras

**Tf-idf** (del inglés Term frequency – Inverse document frequency), frecuencia de término – frecuencia inversa de documento (o sea, la frecuencia de ocurrencia del término en la colección de documentos), es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección.

In [62]:
#Stopwords
spanish_stopwords = stopwords.words('spanish') + list(punctuation)

#stemming
stemmer = SnowballStemmer('spanish')

In [63]:
#Creacion de bolsa de palabras

vectorizer = TfidfVectorizer(sublinear_tf=True       #transformación logarítmica suavizada en frcuencia de término. 
                                                     # ayuda a reducir el impacto de las frecuencias de término altas 
                                                     # y mejora el rendiemiento
                             , min_df=5           #los terminos deben aparecer en almenos 5 documentos
                             , encoding='latin-1'
                             , ngram_range=(1, 3)
                             , stop_words=spanish_stopwords
                       )
features = vectorizer.fit_transform(data.raw)
features.shape

(4472, 8400)

In [64]:
terms = vectorizer.get_feature_names_out()

doc_names = ['{:d}'.format(idx) for idx, _ in enumerate(features)]
dv = pd.DataFrame(data=features.toarray(), index=doc_names, columns=terms)
dv.head()

,00,00 am,00 am 12,00 inscribirte,00 inscribirte debes,00 pm,00 pm 04,00 pm inscribirte,001,001 aws,...,última,última ejecución,última versión,últimas,último,últimos,única,únicamente,único,útil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# Dataset con bolsa de palabras
result = pd.merge(data[['topic']], dv, on=[data[['topic']].index, dv.index],  how="inner")
result.drop(['key_0', 'key_1'], axis='columns', inplace=True)

result[1:40]

,topic_x,00,00 am,00 am 12,00 inscribirte,00 inscribirte debes,00 pm,00 pm 04,00 pm inscribirte,001,...,última,última ejecución,última versión,últimas,último,últimos,única,únicamente,único,útil
1,Preguntas frecuentes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Terms of Service,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Terms of Service,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Privacy Policy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Privacy Policy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Configuración de variables de entorno o de con...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Error en la conexion VPN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,READ ME FIRST: Admin Quick Start Guide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Visualizacion registros de una REDIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,Vulnerabilidad 400 Traceability Loss - AWS - L...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
